In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.8 MB/

In [3]:
import sys
from google.colab import drive
sys.path.append("/content/drive/MyDrive/GenAI2 Final Project/Codes")
from rag_logic import search_products, get_collection_and_model

In [6]:
import random
import numpy as np
from tqdm import tqdm

# Ensure get_collection_and_model() and search_products() exist.

def evaluate_recall_k_single_run(k=5, sample_size=500):
    """
    Performs one evaluation run for Recall@K.
    Randomly samples 'sample_size' items and checks if the exact item
    appears in the top-K results.
    """
    collection, _ = get_collection_and_model()
    all_data = collection.get()
    total_items = len(all_data["ids"])

    if total_items == 0:
        print("❌ Vector DB is empty. Cannot evaluate.")
        return None

    # Select random indices
    sample_size = min(sample_size, total_items)
    random_indices = random.sample(range(total_items), sample_size)

    hits = 0

    for idx in random_indices:
        target_id = all_data["ids"][idx]
        target_title = all_data["metadatas"][idx]["title"]

        results = search_products(query=target_title, top_k=k)
        retrieved_ids = results["ids"][0]

        if target_id in retrieved_ids:
            hits += 1

    return hits / sample_size


def evaluate_recall_k_multi_run(k=5, sample_size=500, runs=3):
    """
    Repeats Recall@K evaluation 'runs' times.
    Calculates mean and standard deviation.
    """
    print(f"🚀 Starting Recall@{k} Evaluation ({sample_size} samples, {runs} runs)...")

    scores = []

    for r in range(runs):
        print(f"\n▶ Run {r+1}/{runs}")
        score = evaluate_recall_k_single_run(k=k, sample_size=sample_size)
        if score is not None:
            scores.append(score)
            print(f"   Run {r+1} Recall@{k}: {score * 100:.2f}%")

    if len(scores) == 0:
        print("❌ No scores collected. Something went wrong.")
        return

    mean_score = np.mean(scores)
    std_score = np.std(scores)

    print("\n================ Final Summary ================")
    print(f"📈 Recall@{k} Mean: {mean_score * 100:.2f}%")
    print(f"📉 Std Dev:        {std_score * 100:.2f}%")
    print("================================================")
    print("Interpretation: Lower std dev means more stable retrieval quality.")

    return scores, mean_score, std_score


if __name__ == "__main__":
    # Example: Recall@5, 500 samples, 3 repeated runs
    evaluate_recall_k_multi_run(k=5, sample_size=500, runs=3)

🚀 Starting Recall@5 Evaluation (500 samples, 3 runs)...

▶ Run 1/3
   Run 1 Recall@5: 86.00%

▶ Run 2/3
   Run 2 Recall@5: 84.40%

▶ Run 3/3
   Run 3 Recall@5: 87.80%

================ Final Summary ================
📈 Recall@5 Mean: 86.07%
📉 Std Dev:        1.39%
Interpretation: Lower std dev means more stable retrieval quality.


In [7]:
# 1. Define queries and 'conditions (price)' as a list of tuples.
custom_queries = [
    ("I need to find a puzzle less than $20", 20.0),
    ("I want to find a stuffed animal for kids less than $30", 30.0),
    ("I want to find a costume for this halloween. Could you recommend for age 8-14?", None)
]

print(f"🚀 Starting Custom Use Case Test ({len(custom_queries)} items)\n")

for i, (query_text, max_price_val) in enumerate(custom_queries):
    print(f"==================================================")
    print(f"🟢 [Query {i+1}]: {query_text}")
    if max_price_val:
        print(f"   👉 Filter Applied: Max Price <= ${max_price_val}")
    print(f"==================================================")

    # 2. Execute Search
    results = search_products(query=query_text, top_k=3, max_price=max_price_val)

    # Parse results (handle safe access)
    if not results['ids'] or not results['ids'][0]:
        print("❌ No search results found.")
        print("\n")
        continue

    ids = results['ids'][0]
    metadatas = results['metadatas'][0]
    documents = results['documents'][0]
    # Handle distances if they exist, otherwise default to 0.0
    distances = results['distances'][0] if results.get('distances') else [0.0] * len(ids)

    for j in range(len(ids)):
        # Extract Metadata
        title = metadatas[j].get("title", "No Title")
        price = metadatas[j].get("price", "N/A")
        rating = metadatas[j].get("rating", "N/A") # Ensure this key matches your DB schema

        # ★ [Modified] Handle case where Document is None
        raw_text = documents[j]
        if raw_text is None:
            display_text = "🚫 [No Content]"
        else:
            display_text = raw_text[:100] + "..." # Truncate to 100 chars

        # 3. Print Results
        print(f"   RANK {j+1} (Distance: {distances[j]:.4f})")
        print(f"   🔹 Title : {title}")
        print(f"   🔹 Price : {price}")

        # Validation Logic
        if max_price_val and isinstance(price, (int, float)):
            if price <= max_price_val:
                print(f"      ✅ Price Condition Met (${price} <= ${max_price_val})")
            else:
                print(f"      ❌ Price Condition Violated! Filtering Failed")

        print(f"   🔹 Rating: {rating}")
        print(f"   🔹 Text  : {display_text}")
        print("-" * 30)
    print("\n")

🚀 Starting Custom Use Case Test (3 items)

🟢 [Query 1]: I need to find a puzzle less than $20
   👉 Filter Applied: Max Price <= $20.0
   RANK 1 (Distance: 0.6427)
   🔹 Title : White Mountain Puzzles Craft Room - 1000 Piece Jigsaw Puzzle
   🔹 Price : 15.99
      ✅ Price Condition Met ($15.99 <= $20.0)
   🔹 Rating: 4.9
   🔹 Text  : Seek and find: for a really good challenge and lots of fun, there are 23 hidden objects for you to s...
------------------------------
   RANK 2 (Distance: 0.6450)
   🔹 Title : White Mountain Puzzles Readers Paradise - 1000 Piece Jigsaw Puzzle
   🔹 Price : 15.99
      ✅ Price Condition Met ($15.99 <= $20.0)
   🔹 Rating: 2.4
   🔹 Text  : Wish upon a book: be immediately drawn into this magical shop with stacks of tomes offering the clas...
------------------------------
   RANK 3 (Distance: 0.6519)
   🔹 Title : MasterPieces Dean Russo Jigsaw Puzzle, Who's a Good Boy? 1000 Pieces
   🔹 Price : 14.99
      ✅ Price Condition Met ($14.99 <= $20.0)
   🔹 Rating: 3.1
 